<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/KNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
treated_data_PB = pd.read_csv('/content/dados_tratados.csv')

### Treinando o modelo KNN

In [ ]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_PB['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution))

marc_pbf
1    65.0
0    35.0
Name: proportion, dtype: float64


In [ ]:
# separar X (features) e y (target)
X = treated_data_PB.drop('marc_pbf', axis=1)
y = treated_data_PB['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_X_resampled = rus.fit_resample(X_train, y_train)

print(y_X_resampled.value_counts())

marc_pbf
0    91196
1    91196
Name: count, dtype: int64


In [ ]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[ 0.51751224 -0.52675384  0.21865512 -0.58552202 -0.49718509 -0.52621268
  -0.50625203 -0.92862846 -0.50777719]
 [ 0.35184965 -0.52675384  0.21865512  2.96187983 -0.49718509 -0.52621268
  -0.50625203 -0.92862846  0.20331283]
 [ 2.77831938 -0.52675384  0.97824874  1.54291909 -0.49718509 -0.52621268
  -0.50625203 -0.92862846 -1.2188672 ]
 [ 0.41356708 -0.52675384  0.21865512  1.54291909  2.01132341  0.56286312
  -0.50625203  1.11409432 -0.50777719]
 [ 1.25812147 -0.52675384  0.21865512 -0.58552202 -0.49718509 -0.52621268
  -0.50625203 -0.92862846 -1.2188672 ]]


In [ ]:
k_values = list(range(1, 31)) # testando valores de 1 a 30

param_grid = {'n_neighbors': k_values}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_scaled, y_X_resampled)

best_k = grid_search.best_params_['n_neighbors']
print("Melhor valor de k encontrado:", best_k)

Melhor valor de k encontrado: 30


### Selecionando features com SelectKBests

In [ ]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_X_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

Características selecionadas: Index(['vlr_renda_media_fam', 'cod_material_piso_fam',
       'cod_agua_canalizada_fam', 'cod_calcamento_domic_fam', 'qtde_pessoas'],
      dtype='object')
